In [72]:
import torch
torch.manual_seed(2618)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import CosineAnnealingLR
from datasets import Dataset
from tqdm import tqdm
import numpy as np
import io
import hashlib


data_path = '/data02/users/lz/code/UICoder/datasets/c4-wash/c4-format-marked/merged'
save_path = '/data02/users/lz/code/UICoder/checkpoints/classifier/c4_test.pth'

In [ ]:
# import os
# from PIL import Image
# from tqdm import tqdm
# train_folder = '/data02/train_data/train'
# dir = '/data02/train_data/data_'
# class_ = 'others'
# dir = dir + class_

# os.makedirs(os.path.join(train_folder,class_),exist_ok=True)
# for idx,filename in enumerate(tqdm(os.listdir(dir))):
#     path = os.path.join(dir,filename)
#     image = Image.open(path)
#     image.save(os.path.join(train_folder,class_,filename))

In [57]:
# def image_to_md5(image):
#     image_bytes = io.BytesIO()
#     image.save(image_bytes, format='PNG')
#     image_data = image_bytes.getvalue()
#     md5_hash = hashlib.md5(image_data)
#     md5_hex = md5_hash.hexdigest()
#     return str(md5_hex)

# os.makedirs(imageFolder,exist_ok=True)
# imageFolder = os.path.join(data_path,'imageFolder-balanced')
# for i in range(0,6):
#     os.makedirs(os.path.join(imageFolder,f'{i}'),exist_ok=True)

# ds = Dataset.load_from_disk(data_path)
# for item in tqdm(ds):
#     item['image'].save(os.path.join(imageFolder,f"{item['score_avr']}/{image_to_md5(item['image'])}.png"))

# for score in range(0,5):
#     for idx,filename in enumerate(os.listdir(os.path.join(imageFolder, f'{score}'))):
#         if idx >= count_min:
#             file_path = os.path.join(os.path.join(imageFolder, f'{score}', filename))
#             os.remove(file_path)

In [80]:
counts = []
for score in range(0,6):
    count = len(os.listdir(os.path.join(os.path.join(data_path,'imageFolder'),f'{score}')))
    counts.append(count)
print(counts)
count_min = min(counts[:5])

[3542, 3840, 1189, 634, 120, 5]


In [101]:
# 定义数据转换
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
inverse_transform = transforms.Compose([
    transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
                         std=[1/0.229, 1/0.224, 1/0.225]),
    transforms.ToPILImage()
])

# 加载完整的数据集
train_dataset = datasets.ImageFolder(os.path.join(data_path,'imageFolder-balanced'), transform=transform)

test_dataset = datasets.ImageFolder(os.path.join(data_path,'imageFolder'), transform=transform)
val_dataset, test_dataset, _ = torch.utils.data.random_split(test_dataset, [500, 500, len(test_dataset)-1000])

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

100%|██████████| 500/500 [00:26<00:00, 43.32it/s]

In [55]:
class2idx = full_dataset.class_to_idx
idx2class = {}
for class_ in class2idx:
    idx2class[class2idx[class_]] = class_ 
print(idx2class)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5'}


In [107]:
# 定义模型
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # 输出为一个评分，因此输出维度为1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 10

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
scheduler = CosineAnnealingLR(optimizer, T_max=38*num_epochs)

# 训练模型
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader,desc="Train"):
        inputs, labels = inputs.to(device), labels.float().to(device)  # 转换为浮点型
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # 计算损失
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs},Running Loss: {epoch_loss:.4f}, End Loss: {loss.item(): 4f}")

    # 在验证集上评估模型
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader,desc="Eval"):
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    print(f"Eval Loss: {val_loss:.4f}\n")

    scheduler.step()

# 保存模型
torch.save(model.state_dict(), save_path)

Train: 100%|██████████| 38/38 [00:13<00:00,  2.74it/s]


Epoch 1/10,Running Loss: 5.2777, End Loss:  16.035381


Eval: 100%|██████████| 32/32 [00:10<00:00,  3.04it/s]


Eval Loss: 1.0366



Train: 100%|██████████| 38/38 [00:13<00:00,  2.77it/s]


...

Eval Loss: 4.9870



Train: 100%|██████████| 38/38 [00:13<00:00,  2.74it/s]


Epoch 10/10,Running Loss: 0.5443, End Loss:  1.811326


Eval: 100%|██████████| 32/32 [00:10<00:00,  3.12it/s]


Eval Loss: 6.2626



In [106]:
# 加载模型
model = models.resnet50()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

model.to(device)
model.load_state_dict(torch.load(save_path))
model.eval()

# 预测并保存结果
predictions = []
with torch.no_grad():
    tbar = tqdm(total=len(test_dataset))
    for inputs, label in test_dataset:
        tbar.update(1)
        outputs = model(torch.stack([inputs]).to(device))
        prediction = int(np.round(outputs.cpu().numpy()[0]))
        predictions.append([label,prediction])  # 假设模型输出是一个 numpy 数组
        
g2 = 0
g2_right = 0
l1 = 0
l1_right = 0
count = 0
allowance = 0
for item in predictions:
    if item[1] < 0:
        item[1] = 0
    elif item[1] > 5:
        item[1] = 5
    if item[0] >= 2:
        g2 += 1
        if abs(item[0] - item[1]) <= allowance:
            g2_right += 1
    if item[0] <= 1:
        l1 += 1
        if abs(item[0] - item[1]) <= allowance:
            l1_right += 1
    if abs(item[0] - item[1]) <= allowance:
        count += 1
print(f'Acc: {round(count/len(predictions),2)}')
print(f'G2 Acc: {round(g2_right/g2,2)}')
print(f'L1 Acc: {round(l1_right/l1,2)}')
print(f'G2L1 Acc: {round((g2_right+l1_right)/(g2+l1),2)}')


# 将预测结果保存到文件
with open('predictions.txt', 'w') as f:
    for pred in predictions:
        f.write(f"{pred}\n")

100%|██████████| 500/500 [01:40<00:00,  4.95it/s]
/tmp/ipykernel_3953953/2819384816.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction = int(np.round(outputs.cpu().numpy()[0]))
100%|█████████▉| 498/500 [00:12<00:00, 46.72it/s]

Acc: 0.38
G2 Acc: 0.0
L1 Acc: 0.47
G2L1 Acc: 0.38
